## Model FAERS Dataset

In [1]:
dataset_id = "faers_modeled"
!bq --location=US mk --dataset {dataset_id}

Dataset 'studied-brand-266702:faers_modeled' successfully created.


In [124]:
# Remove limit on number of columns shown
import pandas as pd
pd.set_option('display.max_columns', None)

### Split Demographic table into separate entities: Patient table, Manufacturer table, Adverse_Event table and Case table 

#### Create Patient table

In [78]:
%%bigquery
create or replace table faers_modeled.Patient as
select ROW_NUMBER() OVER(ORDER BY PRIMARYID) as PATIENT_ID, PRIMARYID as CASE_ID, SEX, AGE, AGE_COD as AGE_UNIT, AGE_GRP as AGE_GROUP, WT as WEIGHT, WT_COD as WEIGHT_UNIT 
from faers_staging.Demographic

""


In [79]:
%%bigquery
select * from faers_modeled.Patient

,PATIENT_ID,CASE_ID,SEX,AGE,AGE_UNIT,AGE_GROUP,WEIGHT,WEIGHT_UNIT
0,351,76194257,M,0,DY,None,1.6,KG
1,406,77767462,M,34,YR,None,NaN,None
2,685,86472573,F,22,YR,A,NaN,None
3,1132,96493314,M,33,YR,None,90.0,KG
4,1188,97790376,F,26,YR,None,NaN,None
...,...,...,...,...,...,...,...,...
394061,363876,157503071,M,84,YR,E,NaN,None
394062,368632,157557171,M,84,YR,None,NaN,None
394063,387912,157786831,F,84,YR,None,89.0,KG
394064,389348,158222071,None,84,YR,None,NaN,None


#### Create Manufacturer table

In [145]:
%%bigquery
create or replace table faers_modeled.Manufacturer as
select ROW_NUMBER() OVER(ORDER BY MFR_SNDR) as MANU_ID, MFR_SNDR as DRUG_MANU 
from faers_staging.Demographic
group by MFR_SNDR

""


In [148]:
%%bigquery
select * from faers_modeled.Manufacturer
order by DRUG_MANU

,MANU_ID,DRUG_MANU
0,1,3M
1,2,AAA PHARMA
2,3,ABBVIE
3,4,ACADIA PHARMACEUTICALS
4,5,ACCESS
...,...,...
466,467,XIROMED
467,468,XTTRIUM
468,469,YUNG SHIN PHARMACEUTICAL
469,470,ZO SKIN HEALTH


#### Create Adverse_Event table by combining with Outcome table

In [134]:
%%bigquery
create or replace table faers_modeled.Adverse_Event as
select ROW_NUMBER() OVER(ORDER BY d.PRIMARYID) as EVENT_ID, d.PRIMARYID as CASE_ID, EVENT_DT_NUM as EVENT_DATE, OCCR_COUNTRY as COUNTRY, OUTC_COD as OUTCOME
from faers_staging.Demographic d left join faers_staging.Outcome o on d.primaryid = o.primaryid

""


In [135]:
%%bigquery
select * from faers_modeled.Adverse_Event

,EVENT_ID,CASE_ID,EVENT_DATE,COUNTRY,OUTCOME
0,315,69273246,2005-08-13,FR,CA
1,846,81808547,None,DE,CA
2,20611,138997085,2017-07-17,AU,CA
3,31037,143464058,2017-06-18,FR,CA
4,41009,146641964,2016-09-24,FR,CA
...,...,...,...,...,...
463265,462830,1474747811,2017-04-15,CZ,OT
463266,462839,1474776311,2018-02-14,JP,OT
463267,463110,1507332710,2017-09-14,FR,OT
463268,463150,1515468211,2018-06-07,US,OT


#### Create Case (Report) table

In [153]:
%%bigquery
create or replace table faers_modeled.Case as
select PRIMARYID as CASE_ID, I_F_CODE as STATUS, REPT_COD as TYPE, REPT_DT_NUM as CASE_DATE, FDA_DT_NUM as FDA_DATE, MFR_DT_NUM as MANU_DATE, MANU_ID, TO_MFR as MANU_NOTIFD, OCCP_COD as REPORTER_OCCP, REPORTER_COUNTRY 
from faers_staging.Demographic d join faers_modeled.Manufacturer m on d.MFR_SNDR = m.DRUG_MANU 

""


In [154]:
%%bigquery
select * from faers_modeled.Case

,CASE_ID,STATUS,TYPE,CASE_DATE,FDA_DATE,MANU_DATE,MANU_ID,MANU_NOTIFD,REPORTER_OCCP,REPORTER_COUNTRY
0,156066741,I,DIR,None,2018-11-06,None,162,N,OT,US
1,156117181,I,DIR,None,2018-11-07,None,162,N,MD,US
2,156809361,I,DIR,None,2018-11-20,None,162,N,OT,US
3,157079191,I,DIR,None,2018-11-28,None,162,N,PH,US
4,157170581,I,DIR,None,2018-11-29,None,162,N,None,US
...,...,...,...,...,...,...,...,...,...,...
394061,155370741,I,DIR,2018-05-05,2018-10-16,None,162,N,CN,US
394062,158618321,I,DIR,2018-05-09,2018-12-31,None,162,N,OT,US
394063,158632541,I,DIR,2018-05-09,2018-12-31,None,162,N,OT,US
394064,155042791,I,DIR,2018-05-10,2018-10-09,None,162,N,CN,US


### Split Drug table into separate entities: Drug table (unique drug records), Active_Ingredient table and Administration (drug dispensing) table 

#### Create Drug table (with unique drug entries)

In [181]:
%%bigquery
create or replace table faers_modeled.Drug as 
select ROW_NUMBER() OVER() as DRUG_ID, DRUGNAME as DRUG_NAME
from faers_staging.Drug
group by DRUGNAME

""


In [187]:
%%bigquery
select * from faers_modeled.Drug
order by DRUG_ID

,DRUG_ID,DRUG_NAME
0,1,ANTIHEMOPHILIC FACTOR
1,2,TAXOTERE
2,3,DIGOXIN.
3,4,PREDNISOLONE.
4,5,PRAVASTATIN.
...,...,...
61839,61840,EFAVIRENZ MYLAN
61840,61841,"RIFADINE IV 600 MG, POUDRE ET SOLVANT POUR SOL..."
61841,61842,LITO
61842,61843,Ofloxacine


#### Create Active_Ingredient table

In [347]:
%%bigquery
create or replace table faers_modeled.Active_Ingredient as 
select ROW_NUMBER() OVER(ORDER BY PROD_AI) as INGREDIENT_ID, PROD_AI as ACTIVE_INGREDIENT
from faers_staging.Drug
group by PROD_AI

""


In [348]:
%%bigquery
select * from faers_modeled.Active_Ingredient

,INGREDIENT_ID,ACTIVE_INGREDIENT
0,109,ACETAMINOPHEN\CAFFEINE CITRATE
1,143,ACETAMINOPHEN\CODEINE\DOXYLAMINE
2,146,ACETAMINOPHEN\DEXTROMETHORPHAN HYDROBROMIDE\DO...
3,153,ACETAMINOPHEN\DEXTROMETHORPHAN HYDROBROMIDE\PS...
4,154,ACETAMINOPHEN\DEXTROMETHORPHAN\PSEUDOEPHEDRINE
...,...,...
5778,5527,TRANYLCYPROMINE SULFATE
5779,5578,TRIMETHOPRIM SULFATE
5780,5623,UMBRALISIB
5781,5691,VILDAGLIPTIN


#### Create Administration (administering of drug) table

In [343]:
%%bigquery
create or replace table faers_modeled.Administration as 
select ROW_NUMBER() OVER(ORDER BY PRIMARYID) as ADMIN_ID, PRIMARYID as CASE_ID, DRUG_ID, INGREDIENT_ID, DRUG_SEQ, ROLE_COD as DRUG_ROLE, LOT_NUM as DRUG_LOT, ROUTE as DRUG_ROUTE, DOSE_FORM, DOSE_FREQ, DOSE_AMT, DOSE_UNIT, CUM_DOSE_CHR as CUM_DOSE, CUM_DOSE_UNIT, DECHAL, RECHAL  
from faers_staging.Drug d left join faers_modeled.Drug d2 on d.DRUGNAME = d2.DRUG_NAME
    left join faers_modeled.Active_Ingredient ai on d.PROD_AI = ai.ACTIVE_INGREDIENT

""


In [344]:
%%bigquery
select * from faers_modeled.Administration

,ADMIN_ID,CASE_ID,DRUG_ID,INGREDIENT_ID,DRUG_SEQ,DRUG_ROLE,DRUG_LOT,DRUG_ROUTE,DOSE_FORM,DOSE_FREQ,DOSE_AMT,DOSE_UNIT,CUM_DOSE,CUM_DOSE_UNIT,DECHAL,RECHAL
0,864,68090726,11649,4481,32,C,None,Unknown,None,None,NaN,None,NaN,None,U,None
1,1345,70848306,69,3723,33,C,None,Oral,None,None,60.0,MG,NaN,None,U,None
2,3022,80189603,3226,4852,34,C,None,Unknown,None,None,NaN,None,NaN,None,U,None
3,3028,80189603,11762,1133,36,C,None,Unknown,None,None,NaN,None,NaN,None,U,None
4,3037,80189603,2507,4994,28,C,None,Intravenous (not otherwise specified),None,None,10.0,ML,NaN,None,U,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1546830,1536501,1427960118,671,2024,21,SS,None,Unknown,None,None,NaN,None,NaN,None,U,None
1546831,1537882,1434453010,1056,4505,21,C,None,None,None,None,NaN,None,NaN,None,D,None
1546832,1538814,1441227316,2588,567,21,SS,UNKNOWN,Unknown,FILM-COATED TABLET,None,5.0,MG,NaN,None,N,None
1546833,1542634,1472865711,3770,2185,21,C,None,Oral,UNKNOWN,None,95.0,MG,NaN,None,None,None


### Generate Primary Key for Reaction table and join with Case table for Foreign Key

In [230]:
%%bigquery
create or replace table faers_modeled.Reaction as 
select ROW_NUMBER() OVER(ORDER BY CASE_ID) as REACTION_ID, CASE_ID, PT as REACTION, DRUG_REC_ACT as RECUR_REACTION
from faers_staging.Reaction r left join faers_modeled.Case c on r.PRIMARYID = c.CASE_ID 

""


In [231]:
%%bigquery
select * from faers_modeled.Reaction

,REACTION_ID,CASE_ID,REACTION,RECUR_REACTION
0,55,39776825,Swelling,None
1,208,59983173,Haematemesis,None
2,233,60789672,Hypotension,None
3,242,61160272,Anuria,None
4,257,61160272,Liver function test increased,None
...,...,...,...,...
1250973,1145651,157561591,Wrong technique in product usage process,None
1250974,1146993,157567781,Wrong technique in product usage process,None
1250975,1148989,157577141,Wrong technique in product usage process,None
1250976,1160363,157628701,Wrong technique in product usage process,None


### Create Diagnosis table from Indication table and join with Administration table for Foreign Key

In [242]:
%%bigquery
create or replace table faers_modeled.Diagnosis as 
select ROW_NUMBER() OVER(ORDER BY ADMIN_ID) as DIAGNOSIS_ID, ADMIN_ID, INDI_PT as DIAGNOSIS
from faers_staging.Indication i left join faers_modeled.Administration a on i.PRIMARYID = a.CASE_ID and i.INDI_DRUG_SEQ = a.DRUG_SEQ 

""


In [243]:
%%bigquery
select * from faers_modeled.Diagnosis

,DIAGNOSIS_ID,ADMIN_ID,DIAGNOSIS
0,224,298,Prophylaxis against graft versus host disease
1,329,430,Sedation
2,397,526,Congestive cardiomyopathy
3,743,1027,Colorectal cancer metastatic
4,892,1239,Renal transplant
...,...,...,...
1064659,1064040,1545857,Product used for unknown indication
1064660,1064076,1545924,Product used for unknown indication
1064661,1064145,1546028,Product used for unknown indication
1064662,1064477,1546544,Product used for unknown indication


### Identify Primary Key (PK) for each modeled table

#### Patient table : PK is PATIENT_ID

In [80]:
%%bigquery
select count(*) as total_records from faers_modeled.Patient

,total_records
0,394066


In [81]:
%%bigquery
select count(distinct PATIENT_ID) as distinct_id from faers_modeled.Patient

,distinct_id
0,394066


#### Manufacturer table : PK is MANU_ID

In [246]:
%%bigquery
select count(*) as total_records from faers_modeled.Manufacturer

,total_records
0,471


In [247]:
%%bigquery
select count(distinct MANU_ID) as distinct_id from faers_modeled.Manufacturer

,distinct_id
0,471


#### Case table : PK is CASE_ID

In [248]:
%%bigquery
select count(*) as total_records from faers_modeled.Case

,total_records
0,394066


In [249]:
%%bigquery
select count(distinct CASE_ID) as distinct_id from faers_modeled.Case

,distinct_id
0,394066


#### Adverse_Event table : PK is EVENT_ID

In [137]:
%%bigquery
select count(*) as total_records from faers_modeled.Adverse_Event

,total_records
0,463270


In [138]:
%%bigquery
select count(distinct EVENT_ID) as distinct_id from faers_modeled.Adverse_Event

,distinct_id
0,463270


#### Drug table : PK is DRUG_ID

In [252]:
%%bigquery
select count(*) as total_records from faers_modeled.Drug

,total_records
0,61844


In [253]:
%%bigquery
select count(distinct DRUG_ID) as distinct_id from faers_modeled.Drug

,distinct_id
0,61844


#### Active_Ingredient table : PK is INGREDIENT_ID

In [254]:
%%bigquery
select count(*) as total_records from faers_modeled.Active_Ingredient

,total_records
0,5783


In [255]:
%%bigquery
select count(distinct INGREDIENT_ID) as distinct_id from faers_modeled.Active_Ingredient

,distinct_id
0,5783


#### Administration table : PK is ADMIN_ID

In [256]:
%%bigquery
select count(*) as total_records from faers_modeled.Administration

,total_records
0,1546835


In [257]:
%%bigquery
select count(distinct ADMIN_ID) as distinct_id from faers_modeled.Administration

,distinct_id
0,1546835


#### Reaction table : PK is REACTION_ID

In [258]:
%%bigquery
select count(*) as total_records from faers_modeled.Reaction

,total_records
0,1250978


In [259]:
%%bigquery
select count(distinct REACTION_ID) as distinct_id from faers_modeled.Reaction

,distinct_id
0,1250978


#### Diagnosis table : PK is DIAGNOSIS_ID

In [260]:
%%bigquery
select count(*) as total_records from faers_modeled.Diagnosis

,total_records
0,1064664


In [261]:
%%bigquery
select count(distinct DIAGNOSIS_ID) as distinct_id from faers_modeled.Diagnosis

,distinct_id
0,1064664


### Check for referential integrity violations 

In [82]:
%%bigquery
select count(*) as count_of_missing_PK
from faers_modeled.Patient p left join faers_modeled.Case c on p.CASE_ID = c.CASE_ID
where c.CASE_ID is null

,count_of_missing_PK
0,0


In [139]:
%%bigquery
select count(*) as count_of_missing_PK
from faers_modeled.Adverse_Event a left join faers_modeled.Case c on a.CASE_ID = c.CASE_ID
where c.CASE_ID is null

,count_of_missing_PK
0,0


In [351]:
%%bigquery
select count(*) as count_of_missing_PK
from faers_modeled.Administration a left join faers_modeled.Case c on a.CASE_ID = c.CASE_ID
where c.CASE_ID is null

,count_of_missing_PK
0,0


In [272]:
%%bigquery
select count(*) as count_of_missing_PK
from faers_modeled.Reaction r left join faers_modeled.Case c on r.CASE_ID = c.CASE_ID
where c.CASE_ID is null

,count_of_missing_PK
0,0


In [263]:
%%bigquery
select count(*) as count_of_missing_PK
from faers_modeled.Case c left join faers_modeled.Manufacturer m on c.MANU_ID = m.MANU_ID
where m.MANU_ID is null

,count_of_missing_PK
0,0


In [271]:
%%bigquery
select count(*) as count_of_missing_PK
from faers_modeled.Diagnosis d left join faers_modeled.Administration a on d.ADMIN_ID = a.ADMIN_ID
where a.ADMIN_ID is null

,count_of_missing_PK
0,0


In [265]:
%%bigquery
select count(*) as count_of_missing_PK
from faers_modeled.Administration a left join faers_modeled.Drug d on a.DRUG_ID = d.DRUG_ID
where d.DRUG_ID is null

,count_of_missing_PK
0,0


In [352]:
%%bigquery
select count(*) as count_of_missing_PK
from faers_modeled.Administration a left join faers_modeled.Active_Ingredient ai on a.INGREDIENT_ID = ai.INGREDIENT_ID
where ai.INGREDIENT_ID is null

,count_of_missing_PK
0,0


## Milestone 10

### Part 1: Data Transformation

#### Transformation 1: Standardize age in unit of years on FAERS Patient table using SQL

##### Step 1: Create intermediate table to convert age in months to years (1 year has 12 months)

In [83]:
%%bigquery
create or replace table faers_modeled.Patient_SQL_1 as
select *, CAST(FLOOR(AGE/12) as INT64) as AGE_YRS 
from faers_modeled.Patient
where AGE_UNIT = 'MON' and AGE is not null 

""


In [84]:
%%bigquery
# Check if transformation done correctly
select * from faers_modeled.Patient_SQL_1 
order by AGE_YRS desc
limit 5

,PATIENT_ID,CASE_ID,SEX,AGE,AGE_UNIT,AGE_GROUP,WEIGHT,WEIGHT_UNIT,AGE_YRS
0,333152,157159021,M,1169,MON,None,NaN,None,97
1,107835,154761773,M,1138,MON,None,67.599998,KG,94
2,174281,155449802,M,1087,MON,None,63.500000,KG,90
3,172311,155429181,F,1087,MON,None,63.500000,KG,90
4,66049,153440393,M,1066,MON,None,63.500000,KG,88


##### Step 2: Create intermediate table to convert age in weeks to years (1 year has approximately 52.14 weeks)

In [85]:
%%bigquery
create or replace table faers_modeled.Patient_SQL_2 as
select *, CAST(FLOOR(AGE/52.14) as INT64) as AGE_YRS 
from faers_modeled.Patient
where AGE_UNIT = 'WK' and AGE is not null 

""


In [86]:
%%bigquery
# Check if transformation done correctly
select * from faers_modeled.Patient_SQL_2 
order by AGE_YRS desc
limit 5

,PATIENT_ID,CASE_ID,SEX,AGE,AGE_UNIT,AGE_GROUP,WEIGHT,WEIGHT_UNIT,AGE_YRS
0,299378,156787721,F,4058,WK,None,108.900002,KG,77
1,311617,156921091,F,3918,WK,None,95.300003,KG,75
2,68655,153618282,F,3708,WK,None,84.800003,KG,71
3,88075,154551062,M,3754,WK,None,99.800003,KG,71
4,72197,153866612,F,3743,WK,None,74.800003,KG,71


##### Step 3: Create intermediate table to convert age in days to years (1 year has 365 days)

In [87]:
%%bigquery
create or replace table faers_modeled.Patient_SQL_3 as
select *, CAST(FLOOR(AGE/365) as INT64) as AGE_YRS 
from faers_modeled.Patient
where AGE_UNIT = 'DY' and AGE is not null 

""


In [88]:
%%bigquery
# Check if transformation done correctly
select * from faers_modeled.Patient_SQL_3 
order by AGE_YRS desc
limit 5

,PATIENT_ID,CASE_ID,SEX,AGE,AGE_UNIT,AGE_GROUP,WEIGHT,WEIGHT_UNIT,AGE_YRS
0,293787,156725961,F,34926,DY,None,63.5,KG,95
1,277787,156552871,F,34465,DY,None,68.0,KG,94
2,372072,157597251,M,33628,DY,None,NaN,None,92
3,207752,155805301,F,33534,DY,None,NaN,None,91
4,58812,152886243,F,33131,DY,None,NaN,None,90


##### Step 4: Create intermediate table to convert age in hours to years (1 year has 365 x 24 hours)

In [89]:
%%bigquery
create or replace table faers_modeled.Patient_SQL_4 as
select *, CAST(FLOOR(AGE/24/365) as INT64) as AGE_YRS 
from faers_modeled.Patient
where AGE_UNIT = 'HR' and AGE is not null 

""


In [90]:
%%bigquery
# Check if transformation done correctly
select * from faers_modeled.Patient_SQL_4
order by AGE_YRS desc
limit 5

,PATIENT_ID,CASE_ID,SEX,AGE,AGE_UNIT,AGE_GROUP,WEIGHT,WEIGHT_UNIT,AGE_YRS
0,291941,156706761,M,12,HR,None,None,None,0
1,291052,156697011,M,62,HR,None,None,None,0
2,294940,156738641,M,60,HR,None,None,None,0
3,86450,154532711,None,3,HR,None,None,None,0
4,89685,154570671,None,3,HR,None,None,None,0


##### Step 5: Create intermediate table to convert age in decades to years (1 decade has 10 years)

In [91]:
%%bigquery
create or replace table faers_modeled.Patient_SQL_5 as
select *, CAST(FLOOR(AGE*10) as INT64) as AGE_YRS 
from faers_modeled.Patient
where AGE_UNIT = 'DEC' and AGE is not null 

""


In [92]:
%%bigquery
# Check if transformation done correctly
select * from faers_modeled.Patient_SQL_5
order by AGE_YRS desc
limit 5

,PATIENT_ID,CASE_ID,SEX,AGE,AGE_UNIT,AGE_GROUP,WEIGHT,WEIGHT_UNIT,AGE_YRS
0,333021,157157552,F,10,DEC,None,None,None,100
1,22358,143393573,F,10,DEC,None,None,None,100
2,297989,156772542,F,10,DEC,None,None,None,100
3,214240,155873141,None,10,DEC,None,None,None,100
4,289439,156678922,None,10,DEC,None,None,None,100


##### Step 6: Create final table based on Patient_SQL_1 + Patient_SQL_2 + Patient_SQL_3 + Patient_SQL_4 + Patient_SQL_5 + age in years & null ages from original Patient table 

In [93]:
%%bigquery
create or replace table faers_modeled.Patient_SQL_Final as
(select PATIENT_ID, CASE_ID, SEX, AGE_YRS, AGE_GROUP, WEIGHT, WEIGHT_UNIT
 from faers_modeled.Patient_SQL_1
union all
 select PATIENT_ID, CASE_ID, SEX, AGE_YRS, AGE_GROUP, WEIGHT, WEIGHT_UNIT
 from faers_modeled.Patient_SQL_2
union all
 select PATIENT_ID, CASE_ID, SEX, AGE_YRS, AGE_GROUP, WEIGHT, WEIGHT_UNIT
 from faers_modeled.Patient_SQL_3
union all
 select PATIENT_ID, CASE_ID, SEX, AGE_YRS, AGE_GROUP, WEIGHT, WEIGHT_UNIT
 from faers_modeled.Patient_SQL_4
union all
 select PATIENT_ID, CASE_ID, SEX, AGE_YRS, AGE_GROUP, WEIGHT, WEIGHT_UNIT
 from faers_modeled.Patient_SQL_5
union all
 select PATIENT_ID, CASE_ID, SEX, AGE as AGE_YRS, AGE_GROUP, WEIGHT, WEIGHT_UNIT
 from faers_modeled.Patient
 where AGE is null or AGE_UNIT = 'YR'
)

""


In [94]:
%%bigquery
# Check if union done correctly
select * from faers_modeled.Patient_SQL_Final
order by AGE_YRS desc
limit 5

,PATIENT_ID,CASE_ID,SEX,AGE_YRS,AGE_GROUP,WEIGHT,WEIGHT_UNIT
0,82577,154478603,M,117,None,NaN,None
1,169467,155399611,M,108,None,56.0,KG
2,272593,156497401,F,107,None,NaN,None
3,292547,156713241,F,107,None,NaN,None
4,267958,156447341,F,107,E,NaN,None


##### Step 7: Check if number of records before and after transformation match to ensure all records retained

In [95]:
%%bigquery
# Check number of rows of final table
select count(*) as number_of_final_records from faers_modeled.Patient_SQL_Final

,number_of_final_records
0,394066


In [96]:
%%bigquery
# Check number of rows of initial table
select count(*) as number_of_initial_records from faers_modeled.Patient

,number_of_initial_records
0,394066


##### Only 1 data transformation needed as the other two cross-dataset queries can be performed without any transformation.

### Part 2: Verify BigQuery tables have valid Primary Key and Foreign Key 

#### Patient_SQL_Final table : PK is PATIENT_ID

In [97]:
%%bigquery
select count(*) as total_records from faers_modeled.Patient_SQL_Final

,total_records
0,394066


In [98]:
%%bigquery
select count(distinct PATIENT_ID) as distinct_id from faers_modeled.Patient_SQL_Final

,distinct_id
0,394066


#### Patient_SQL_Final table : FK is CASE_ID

In [99]:
%%bigquery
select count(*) as count_of_missing_PK
from faers_modeled.Patient_SQL_Final p left join faers_modeled.Case c on p.CASE_ID = c.CASE_ID
where c.CASE_ID is null

,count_of_missing_PK
0,0
